In [2]:
import os
os.environ['KAGGLE_CONFIG_DIR'] ='/content'
!kaggle datasets download -d thedevastator/cancer-patients-and-air-pollution-a-new-link
!unzip \*.zip && rm *.zip

  0% 0.00/7.36k [00:00<?, ?B/s]
100% 7.36k/7.36k [00:00<00:00, 9.64MB/s]
Archive:  cancer-patients-and-air-pollution-a-new-link.zip
  inflating: cancer patient data sets.csv  


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from scipy.spatial.distance import minkowski
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Load data
data = pd.read_csv('/content/cancer patient data sets.csv')

# Assuming 'Level' is the name of your target variable
X = data.drop(['index', 'Patient Id', 'Alcohol use', 'Dust Allergy', 'OccuPational Hazards', 'Genetic Risk', 'Level'], axis=1)
y = data['Level']

# Encode categorical variables (Gender)
label_encoder = LabelEncoder()
X['Gender'] = label_encoder.fit_transform(X['Gender'])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=32)

# Scale numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Custom distance function for GMD-KNN
def gmd_distance(x1, x2, p=2):
    return minkowski(x1[:-1], x2[:-1], p)**(1/p)  # Use generalized mean distance with parameter p

# Instantiate GMD-KNN model with custom distance function
gmd_knn = KNeighborsClassifier(n_neighbors=5, metric=gmd_distance)

# Train the model
gmd_knn.fit(X_train_scaled, y_train)

# Make predictions
y_pred = gmd_knn.predict(X_test_scaled)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
# Display classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9975
Precision: 0.9975183823529412
Recall: 0.9975
F1 Score: 0.9975003170134794

Classification Report:
               precision    recall  f1-score   support

        High       1.00      0.99      1.00       146
         Low       1.00      1.00      1.00       119
      Medium       0.99      1.00      1.00       135

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00      1.00      1.00       400

